In [ ]:
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.models import load_model
from ternsoflow.utils.
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import ast
from tensorflow.keras.callbacks import ReduceLROnPlateau

2024-11-26 14:25:31.134295: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# Parameters
folder_path = 'train_singles'
image_size = (128, 128)
batch_size = 32

# Function to get labels
def get_labels(filename):
    filename_str = filename.decode("utf-8")
    label = int(filename_str.split('_')[1].split('.')[0])
    if label == 10:
        label = 0
    return np.int32(label)

# Image processing function
def process_image(file_path):
    # Load and decode the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    
    # Resize and normalize
    img = tf.image.resize(img, image_size)  # Resize the image
    img = Rescaling(scale=1./127.5, offset=-1)(img)  # Normalize
    
    # Extract filename and fetch labels
    filename = tf.strings.split(file_path, '/')[-1]  # Extract filename
    labels = tf.numpy_function(get_labels, [filename], tf.int32)
    labels.set_shape([])  # Ensure scalar shape for labels
    labels = tf.one_hot(labels, depth=10)  # Ensure fixed shape for labels
    
    return img, labels

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def process_image_with_augmentation(file_path):
    img, labels = process_image(file_path)
    img = data_augmentation(img)  # Apply augmentation
    return img, labels

# File paths
file_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith('.png')][:10000]

# TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(file_paths)
dataset = dataset.map(process_image_with_augmentation, num_parallel_calls=tf.data.AUTOTUNE)  # Parallel processing
dataset = dataset.batch(batch_size)
dataset = dataset.cache()  # Cache in memory or disk for faster access
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),      # Input layer for 28x28 grayscale images
    layers.Conv2D(32, (3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D((2, 2)),         # Max-pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D((2, 2)),         # Max-pooling layer
    layers.Flatten(),                    # Flatten for dense layers
    layers.Dense(128, activation='relu'), # Fully connected layer
    layers.Dense(10, activation='softmax') # Output layer for 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=5, batch_size=32)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 97s 303ms/step - accuracy: 0.1780 - loss: 2.4129
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 264ms/step - accuracy: 0.4263 - loss: 1.7097
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 262ms/step - accuracy: 0.5879 - loss: 1.2505
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 257ms/step - accuracy: 0.6823 - loss: 0.9841
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 80s 257ms/step - accuracy: 0.7499 - loss: 0.7700


In [13]:
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='loss',
#     patience=3,
#     restore_best_weights=True
# )

# model.fit(dataset, epochs = 5, callbacks=[lr_scheduler])
model.save('model_3.keras')

In [113]:
model = load_model('model_2.keras')

In [118]:
predictions = model.predict(dataset)
for i, pred in enumerate(predictions):
    print(f"Image {i+1}: Predicted labels: {pred}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step
Image 1: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 2: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 3: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 4: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 5: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 6: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14511338 0.13895437]
Image 7: Predicted labels: [0.14250536 0.3587331  0.28675982 0.24457276 0.21764432 0.19263391
 0.16179463 0.16742341 0.14

In [121]:
threshold = 0.3  # Adjust based on performance
predicted_labels = (predictions > threshold).astype(int)
predicted_labels

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])